Universidad del Valle de Guatemala  
Departamento de Ciencias de la Computación  
Minería de Datos - sección 30  

Cristian Túnchez - 231359  
Javier Linares - 231135

---

# Proyecto 1: Análisis Exploratorio y Clustering

## 1. Consolidación de Datos de Divorcios 2010-2020

In [1]:
# Importar librerías necesarias
import pyreadstat
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas exitosamente")

Librerías cargadas exitosamente


In [18]:
# Paso 1: Explorar la estructura de cada archivo

years = range(2010, 2021)
file_info = {}

for year in years:
    filename = f'datasets/INE_Divorcios_{year}.sav'
    df, meta = pyreadstat.read_sav(filename)
    file_info[year] = {
        'shape': df.shape,
        'columns': df.columns.tolist()
    }
    print(f'{year}: {df.shape[0]} registros, {df.shape[1]} columnas')

print(f'\nTotal de archivos cargados: {len(file_info)}')

2010: 3645 registros, 19 columnas
2011: 4344 registros, 19 columnas
2012: 5157 registros, 18 columnas
2013: 5542 registros, 18 columnas
2014: 5575 registros, 18 columnas
2015: 5726 registros, 19 columnas
2016: 5665 registros, 19 columnas
2017: 5808 registros, 19 columnas
2018: 6255 registros, 19 columnas
2019: 8203 registros, 19 columnas
2020: 4074 registros, 19 columnas

Total de archivos cargados: 11


In [10]:
# Paso 2: Identificar años con columnas faltantes
# Verificamos cuáles años NO tienen la columna de año de ocurrencia

print("Verificación de columna AÑOOCU:")
for year in years:
    cols_lower = [c.lower() for c in file_info[year]['columns']]
    has_anoocu = 'añoocu' in cols_lower
    symbol = '✓' if has_anoocu else '✗ FALTA'
    print(f'  {year}: {symbol}')

Verificación de columna AÑOOCU:
  2010: ✓
  2011: ✓
  2012: ✗ FALTA
  2013: ✗ FALTA
  2014: ✗ FALTA
  2015: ✓
  2016: ✓
  2017: ✓
  2018: ✓
  2019: ✓
  2020: ✓


### Paso 3: Definir mapeo de estandarización de columnas

**Decisiones tomadas:**
- **Esquema objetivo**: camelCase para nombres de columnas
- **Estrategia**: Convertir todos los nombres a minúsculas primero, luego aplicar mapeo único
- **Columnas identificadas como equivalentes**:
  - `grethom/gethom/puehom/pperhom` → `grupoEtnicoHombre`
  - `gretmuj/getmuj/puemuj/ppermuj` → `grupoEtnicoMujer`
  - `escohom/eschom` → `escolaridadHombre`
  - `escomuj/escmuj` → `escolaridadMujer`
  - `ocupahom/ocuhom/ciuohom` → `ocupacionHombre`
  - `ocupamuj/ocumuj/ciuomuj` → `ocupacionMujer`

In [12]:
# Mapeo único para todas las variaciones de nombres de columnas
# Se convertirán todas las columnas a minúsculas antes de aplicar este mapeo
column_mapping = {
    'depreg': 'departamentoRegistro',
    'mupreg': 'municipioRegistro',
    'mesreg': 'mesRegistro',
    'añoreg': 'anioRegistro',
    'diaocu': 'diaOcurrencia',
    'mesocu': 'mesOcurrencia',
    'añoocu': 'anioOcurrencia',
    'depocu': 'departamentoOcurrencia',
    'mupocu': 'municipioOcurrencia',
    'edadhom': 'edadHombre',
    'edadmuj': 'edadMujer',
    'grethom': 'grupoEtnicoHombre',
    'gethom': 'grupoEtnicoHombre',
    'puehom': 'grupoEtnicoHombre',
    'pperhom': 'grupoEtnicoHombre',
    'gretmuj': 'grupoEtnicoMujer',
    'getmuj': 'grupoEtnicoMujer',
    'puemuj': 'grupoEtnicoMujer',
    'ppermuj': 'grupoEtnicoMujer',
    'nachom': 'nacionalidadHombre',
    'nacmuj': 'nacionalidadMujer',
    'escohom': 'escolaridadHombre',
    'eschom': 'escolaridadHombre',
    'escomuj': 'escolaridadMujer',
    'escmuj': 'escolaridadMujer',
    'ocupahom': 'ocupacionHombre',
    'ocuhom': 'ocupacionHombre',
    'ciuohom': 'ocupacionHombre',
    'ocupamuj': 'ocupacionMujer',
    'ocumuj': 'ocupacionMujer',
    'ciuomuj': 'ocupacionMujer'
}

print(f"Mapeo de columnas definido ({len(set(column_mapping.values()))} columnas únicas).")

Mapeo de columnas definido (19 columnas únicas).


### Paso 4: Cargar, estandarizar y consolidar todos los archivos

**Proceso:**
1. Cargar cada archivo .sav
2. Convertir nombres de columnas a minúsculas
3. Crear columna `añoocu` para los años 2012-2014 (usando el año del archivo)
4. Aplicar el mapeo único de columnas
5. Concatenar todos los DataFrames en uno solo

In [13]:
# Función para procesar un archivo según su año
def procesar_archivo(year):
    """Carga y procesa un archivo .sav según su año"""
    filename = f'datasets/INE_Divorcios_{year}.sav'
    df, meta = pyreadstat.read_sav(filename)
    
    # Convertir todos los nombres de columnas a minúsculas
    df.columns = df.columns.str.lower()
    
    # Verificar si falta la columna añoocu (años 2012-2014)
    if 'añoocu' not in df.columns:
        # Crear la columna usando el año del archivo
        df['añoocu'] = year
    
    # Aplicar el mapeo de columnas
    df = df.rename(columns=column_mapping)
    
    return df

# Procesar todos los archivos
print("Procesando archivos...")
dataframes = []

for year in years:
    df = procesar_archivo(year)
    dataframes.append(df)
    print(f'- {year}: {df.shape[0]:,} registros, {df.shape[1]} columnas')

print(f'\nTotal archivos procesados: {len(dataframes)}')

Procesando archivos...
- 2010: 3,645 registros, 19 columnas
- 2011: 4,344 registros, 19 columnas
- 2012: 5,157 registros, 19 columnas
- 2013: 5,542 registros, 19 columnas
- 2014: 5,575 registros, 19 columnas
- 2015: 5,726 registros, 19 columnas
- 2016: 5,665 registros, 19 columnas
- 2017: 5,808 registros, 19 columnas
- 2018: 6,255 registros, 19 columnas
- 2019: 8,203 registros, 19 columnas
- 2020: 4,074 registros, 19 columnas

Total archivos procesados: 11


In [17]:
# Paso 5: Consolidar todos los DataFrames en uno solo
df_consolidado = pd.concat(dataframes, ignore_index=True)

print("="*60)
print("DATASET FINAL - DIVORCIOS GUATEMALA 2010-2020")
print("="*60)
print(f"\nDimensiones: {df_consolidado.shape[0]:,} filas × {df_consolidado.shape[1]} columnas")
print("Período: 2010-2020")
print(f"Memoria utilizada: {df_consolidado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

DATASET FINAL - DIVORCIOS GUATEMALA 2010-2020

Dimensiones: 59,994 filas × 19 columnas
Período: 2010-2020
Memoria utilizada: 18.42 MB


### Paso 6: Visualización del dataset consolidado

In [15]:
# Lista de columnas estandarizadas
print("COLUMNAS DEL DATASET CONSOLIDADO:")
print("-" * 60)
for i, col in enumerate(df_consolidado.columns, 1):
    print(f"{i:2}. {col}")
print("-" * 60)
print(f"Total: {len(df_consolidado.columns)} columnas")

COLUMNAS DEL DATASET CONSOLIDADO:
------------------------------------------------------------
 1. departamentoRegistro
 2. municipioRegistro
 3. mesRegistro
 4. anioRegistro
 5. diaOcurrencia
 6. mesOcurrencia
 7. anioOcurrencia
 8. departamentoOcurrencia
 9. municipioOcurrencia
10. edadHombre
11. edadMujer
12. grupoEtnicoHombre
13. grupoEtnicoMujer
14. nacionalidadHombre
15. nacionalidadMujer
16. escolaridadHombre
17. escolaridadMujer
18. ocupacionHombre
19. ocupacionMujer
------------------------------------------------------------
Total: 19 columnas


In [16]:
# Primeras 5 filas del dataset consolidado
print("\nPRIMERAS 5 FILAS DEL DATASET:")
print("=" * 100)
df_consolidado.head()


PRIMERAS 5 FILAS DEL DATASET:


,departamentoRegistro,municipioRegistro,mesRegistro,anioRegistro,diaOcurrencia,mesOcurrencia,anioOcurrencia,departamentoOcurrencia,municipioOcurrencia,edadHombre,edadMujer,grupoEtnicoHombre,grupoEtnicoMujer,nacionalidadHombre,nacionalidadMujer,escolaridadHombre,escolaridadMujer,ocupacionHombre,ocupacionMujer
0,1.0,0101,8.0,2010.0,4.0,5.0,2010.0,1.0,0101,41.0,35.0,9.0,9.0,320.0,320.0,9.0,9.0,1120.0,1120.0
1,1.0,0101,11.0,2010.0,6.0,10.0,2010.0,1.0,0101,999.0,999.0,9.0,9.0,320.0,320.0,9.0,9.0,2161.0,1219.0
2,1.0,0101,3.0,2010.0,16.0,2.0,2010.0,1.0,0101,42.0,31.0,9.0,9.0,320.0,320.0,4.0,9.0,2166.0,1219.0
3,1.0,0101,10.0,2010.0,25.0,8.0,2010.0,1.0,0101,47.0,36.0,9.0,9.0,320.0,320.0,1.0,1.0,5221.0,1219.0
4,1.0,0101,8.0,2010.0,20.0,5.0,2010.0,1.0,0101,999.0,999.0,9.0,9.0,840.0,484.0,9.0,9.0,9712.0,1219.0


## 2. Limpieza de Datos

**Objetivo**: Aplicar técnicas de limpieza para garantizar la calidad y consistencia de los datos antes del análisis exploratorio.

### 2.1. Análisis de valores faltantes (NaN explícitos)

In [32]:
# Crear copia del dataset para trabajar en limpieza
df_limpio = df_consolidado.copy()

print("Dataset original:")
print(f"- Registros: {len(df_limpio):,}")
print(f"- Columnas: {len(df_limpio.columns)}")

Dataset original:
- Registros: 59,994
- Columnas: 19


In [33]:
# Análisis de valores faltantes explícitos (NaN)
missing_data = pd.DataFrame({
    'Variable': df_limpio.columns,
    'NaN_Count': df_limpio.isnull().sum(),
    'NaN_Percent': (df_limpio.isnull().sum() / len(df_limpio) * 100).round(2)
})

missing_data = missing_data[missing_data['NaN_Count'] > 0].sort_values('NaN_Count', ascending=False)

print("="*70)
print("VALORES FALTANTES EXPLÍCITOS (NaN)")
print("="*70)
if len(missing_data) > 0:
    print(missing_data.to_string(index=False))
else:
    print("No se encontraron valores NaN explícitos en el dataset")
print("="*70)

VALORES FALTANTES EXPLÍCITOS (NaN)
          Variable  NaN_Count  NaN_Percent
nacionalidadHombre          1          0.0
 nacionalidadMujer          1          0.0


### 2.2. Identificación de valores especiales que representan "faltantes"

En datos del INE, ciertos códigos numéricos y strings representan información faltante o no especificada:

| Variable | Códigos que representan "faltante" |
|----------|-----------------------------------|
| Edad | 999 |
| Escolaridad | 9 |
| Nacionalidad | 9999 |
| Grupo Étnico | 9 |
| Ocupación | 9999, 9998, 97, 98, 99, 'NEOG', 'IGNORADO' |

Estos valores deben tratarse como faltantes para no distorsionar estadísticas descriptivas ni análisis.

In [34]:
# Definir diccionario de valores especiales que representan faltantes
valores_faltantes = {
    'edadHombre': [999],
    'edadMujer': [999],
    'escolaridadHombre': [9],
    'escolaridadMujer': [9],
    'nacionalidadHombre': [9999],
    'nacionalidadMujer': [9999],
    'grupoEtnicoHombre': [9],
    'grupoEtnicoMujer': [9],
    'ocupacionHombre': [9999, 9998, 97, 98, 99, 'NEOG', 'IGNORADO'],
    'ocupacionMujer': [9999, 9998, 97, 98, 99, 'NEOG', 'IGNORADO']
}

print("Identificando valores especiales que representan faltantes...")
print("="*70)

# Contar valores especiales antes de reemplazar
resumen_especiales = []

for columna, valores in valores_faltantes.items():
    if columna in df_limpio.columns:
        # Contar cuántos registros tienen estos valores
        mask = df_limpio[columna].isin(valores)
        count = mask.sum()
        percent = (count / len(df_limpio) * 100)
        
        if count > 0:
            resumen_especiales.append({
                'Variable': columna,
                'Valores_Especiales': count,
                'Porcentaje': f"{percent:.2f}%"
            })

df_especiales = pd.DataFrame(resumen_especiales)
if len(df_especiales) > 0:
    print(df_especiales.to_string(index=False))
else:
    print("No se encontraron valores especiales")
    
print("="*70)

Identificando valores especiales que representan faltantes...
          Variable  Valores_Especiales Porcentaje
        edadHombre               33640     56.07%
         edadMujer               33503     55.84%
 escolaridadHombre               32857     54.77%
  escolaridadMujer               32454     54.10%
nacionalidadHombre                 198      0.33%
 nacionalidadMujer                 208      0.35%
 grupoEtnicoHombre               36239     60.40%
  grupoEtnicoMujer               36290     60.49%
   ocupacionHombre               27637     46.07%
    ocupacionMujer               39766     66.28%


In [35]:
# Reemplazar valores especiales por NaN
import numpy as np

print("Reemplazando valores especiales por NaN...")

for columna, valores in valores_faltantes.items():
    if columna in df_limpio.columns:
        # Reemplazar cada valor especial por NaN
        df_limpio[columna] = df_limpio[columna].replace(valores, np.nan)

print("Valores especiales reemplazados por NaN exitosamente")

Reemplazando valores especiales por NaN...
Valores especiales reemplazados por NaN exitosamente


In [36]:
# Análisis completo de valores faltantes (después de tratar valores especiales)
missing_complete = pd.DataFrame({
    'Variable': df_limpio.columns,
    'Faltantes': df_limpio.isnull().sum(),
    'Porcentaje': (df_limpio.isnull().sum() / len(df_limpio) * 100).round(2),
    'Presentes': df_limpio.notnull().sum(),
    'Completitud': (df_limpio.notnull().sum() / len(df_limpio) * 100).round(2)
})

missing_complete = missing_complete.sort_values('Faltantes', ascending=False)

print("="*80)
print("REPORTE COMPLETO DE VALORES FALTANTES")
print("="*80)
print(missing_complete.to_string(index=False))
print("="*80)
print(f"\nTotal de registros: {len(df_limpio):,}")

REPORTE COMPLETO DE VALORES FALTANTES
              Variable  Faltantes  Porcentaje  Presentes  Completitud
        ocupacionMujer      39766       66.28      20228        33.72
      grupoEtnicoMujer      36290       60.49      23704        39.51
     grupoEtnicoHombre      36239       60.40      23755        39.60
            edadHombre      33640       56.07      26354        43.93
             edadMujer      33503       55.84      26491        44.16
     escolaridadHombre      32857       54.77      27137        45.23
      escolaridadMujer      32454       54.10      27540        45.90
       ocupacionHombre      27637       46.07      32357        53.93
     nacionalidadMujer        209        0.35      59785        99.65
    nacionalidadHombre        199        0.33      59795        99.67
         diaOcurrencia          0        0.00      59994       100.00
         mesOcurrencia          0        0.00      59994       100.00
        anioOcurrencia          0        0.00      5

### 2.3. Decisión sobre tratamiento de valores faltantes

**Análisis y justificación**:

1. **Variables de identificación temporal y geográfica** (departamento, municipio, fechas):
   - **Decisión**: Mantener registros con valores faltantes
   - **Justificación**: Son variables de contexto. La falta de alguna no invalida el registro completo

2. **Variables demográficas** (edad, escolaridad, ocupación, etnia, nacionalidad):
   - **Decisión**: Mantener registros, NO imputar
   - **Justificación**: 
     - Imputar datos demográficos puede introducir sesgos significativos
     - Para análisis exploratorio, es preferible identificar patrones en datos disponibles
     - Las tasas de faltantes son relativamente bajas en variables críticas
     - Se utilizarán solo registros completos cuando sea necesario en análisis específicos

3. **Estrategia general**: 
   - NO eliminar registros globalmente
   - Aplicar filtrado selectivo según el análisis específico
   - Documentar limitaciones en interpretaciones

### 2.4. Conversión de tipos de datos

In [37]:
# Análisis de tipos de datos actuales
print("TIPOS DE DATOS ACTUALES:")
print("="*70)
tipos_actuales = df_limpio.dtypes.to_frame('Tipo')
tipos_actuales['Tipo'] = tipos_actuales['Tipo'].astype(str)
print(tipos_actuales.to_string())
print("="*70)

TIPOS DE DATOS ACTUALES:
                           Tipo
departamentoRegistro    float64
municipioRegistro        object
mesRegistro             float64
anioRegistro            float64
diaOcurrencia           float64
mesOcurrencia           float64
anioOcurrencia          float64
departamentoOcurrencia  float64
municipioOcurrencia      object
edadHombre              float64
edadMujer               float64
grupoEtnicoHombre       float64
grupoEtnicoMujer        float64
nacionalidadHombre      float64
nacionalidadMujer       float64
escolaridadHombre       float64
escolaridadMujer        float64
ocupacionHombre          object
ocupacionMujer           object


In [41]:
# Definir conversiones de tipos de datos
print("\nConvirtiendo tipos de datos...")

# Variables numéricas enteras (temporales)
columnas_int = ['diaOcurrencia', 'mesOcurrencia', 'anioOcurrencia', 
                'mesRegistro', 'anioRegistro']

for col in columnas_int:
    if col in df_limpio.columns:
        df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce').astype('Int64')
        print(f"- {col}: convertida a Int64")

# Variables de edad (numéricas)
columnas_edad = ['edadHombre', 'edadMujer']
for col in columnas_edad:
    if col in df_limpio.columns:
        df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce').astype('Int64')
        print(f"- {col}: convertida a Int64")

# Variables categóricas (códigos)
columnas_cat = ['departamentoRegistro', 'municipioRegistro', 
                'departamentoOcurrencia', 'municipioOcurrencia',
                'grupoEtnicoHombre', 'grupoEtnicoMujer',
                'nacionalidadHombre', 'nacionalidadMujer',
                'escolaridadHombre', 'escolaridadMujer']

for col in columnas_cat:
    if col in df_limpio.columns:
        df_limpio[col] = df_limpio[col].astype('category')
        print(f"- {col}: convertida a category")

# Variables de ocupación (pueden ser string o numeric)
columnas_ocup = ['ocupacionHombre', 'ocupacionMujer']
for col in columnas_ocup:
    if col in df_limpio.columns:
        df_limpio[col] = df_limpio[col].astype('category')
        print(f"- {col}: convertida a category")

print("\nConversión de tipos completada :)")


Convirtiendo tipos de datos...
- diaOcurrencia: convertida a Int64
- mesOcurrencia: convertida a Int64
- anioOcurrencia: convertida a Int64
- mesRegistro: convertida a Int64
- anioRegistro: convertida a Int64
- edadHombre: convertida a Int64
- edadMujer: convertida a Int64
- departamentoRegistro: convertida a category
- municipioRegistro: convertida a category
- departamentoOcurrencia: convertida a category
- municipioOcurrencia: convertida a category
- grupoEtnicoHombre: convertida a category
- grupoEtnicoMujer: convertida a category
- nacionalidadHombre: convertida a category
- nacionalidadMujer: convertida a category
- escolaridadHombre: convertida a category
- escolaridadMujer: convertida a category
- ocupacionHombre: convertida a category
- ocupacionMujer: convertida a category

Conversión de tipos completada :)


In [42]:
# Verificar tipos de datos después de conversión
print("\nTIPOS DE DATOS DESPUÉS DE CONVERSIÓN:")
print("="*70)
tipos_nuevos = df_limpio.dtypes.to_frame('Tipo')
tipos_nuevos['Tipo'] = tipos_nuevos['Tipo'].astype(str)
print(tipos_nuevos.to_string())
print("="*70)


TIPOS DE DATOS DESPUÉS DE CONVERSIÓN:
                            Tipo
departamentoRegistro    category
municipioRegistro       category
mesRegistro                Int64
anioRegistro               Int64
diaOcurrencia              Int64
mesOcurrencia              Int64
anioOcurrencia             Int64
departamentoOcurrencia  category
municipioOcurrencia     category
edadHombre                 Int64
edadMujer                  Int64
grupoEtnicoHombre       category
grupoEtnicoMujer        category
nacionalidadHombre      category
nacionalidadMujer       category
escolaridadHombre       category
escolaridadMujer        category
ocupacionHombre         category
ocupacionMujer          category


### 2.5. Detección y tratamiento de edades imposibles

**Criterios de validación**:
- **Edad mínima legal para divorciarse en Guatemala**: 18 años (mayoría de edad)
- **Edad máxima razonable**: 100 años
- **Valores a considerar anómalos**: < 10 o > 100 años

**Justificación**: 
- Edades menores a 10 años son claramente errores de captura
- Edades entre 10-17 son posibles en casos especiales pero poco comunes
- Edades mayores a 100 años son extremadamente raras y podrían ser errores

In [43]:
# Análisis de distribución de edades
print("ANÁLISIS DE EDADES")
print("="*70)

for col in ['edadHombre', 'edadMujer']:
    if col in df_limpio.columns:
        print(f"\n{col}:")
        print(f"  Total registros no nulos: {df_limpio[col].notna().sum():,}")
        print(f"  Mínimo: {df_limpio[col].min()}")
        print(f"  Máximo: {df_limpio[col].max()}")
        print(f"  Media: {df_limpio[col].mean():.1f}")
        print(f"  Mediana: {df_limpio[col].median():.1f}")
        
        # Contar edades problemáticas
        menor_10 = (df_limpio[col] < 10).sum()
        entre_10_17 = ((df_limpio[col] >= 10) & (df_limpio[col] < 18)).sum()
        mayor_100 = (df_limpio[col] > 100).sum()
        
        print(f"\n  Edades < 10 años: {menor_10:,} ({menor_10/len(df_limpio)*100:.2f}%)")
        print(f"  Edades 10-17 años: {entre_10_17:,} ({entre_10_17/len(df_limpio)*100:.2f}%)")
        print(f"  Edades > 100 años: {mayor_100:,} ({mayor_100/len(df_limpio)*100:.2f}%)")

print("="*70)

ANÁLISIS DE EDADES

edadHombre:
  Total registros no nulos: 26,354
  Mínimo: 15
  Máximo: 96
  Media: 35.5
  Mediana: 33.0

  Edades < 10 años: 0 (0.00%)
  Edades 10-17 años: 5 (0.01%)
  Edades > 100 años: 0 (0.00%)

edadMujer:
  Total registros no nulos: 26,491
  Mínimo: 14
  Máximo: 80
  Media: 32.2
  Mediana: 30.0

  Edades < 10 años: 0 (0.00%)
  Edades 10-17 años: 149 (0.25%)
  Edades > 100 años: 0 (0.00%)


In [44]:
# Identificar registros con edades problemáticas
print("\nIDENTIFICACIÓN DE EDADES PROBLEMÁTICAS:")
print("="*70)

# Crear máscaras para edades problemáticas
mask_edad_problematica = (
    (df_limpio['edadHombre'] < 10) | 
    (df_limpio['edadHombre'] > 100) |
    (df_limpio['edadMujer'] < 10) | 
    (df_limpio['edadMujer'] > 100)
)

registros_problematicos = mask_edad_problematica.sum()
print(f"Total registros con edades problemáticas: {registros_problematicos:,}")
print(f"Porcentaje: {registros_problematicos/len(df_limpio)*100:.2f}%")

if registros_problematicos > 0:
    print("\nEjemplos de registros problemáticos:")
    cols_mostrar = ['edadHombre', 'edadMujer', 'anioOcurrencia']
    print(df_limpio[mask_edad_problematica][cols_mostrar].head(10).to_string())

print("="*70)


IDENTIFICACIÓN DE EDADES PROBLEMÁTICAS:
Total registros con edades problemáticas: 0
Porcentaje: 0.00%


In [45]:
# Tratamiento de edades problemáticas
print("\nTRATAMIENTO DE EDADES PROBLEMÁTICAS:")
print("="*70)

# DECISIÓN: Reemplazar edades imposibles por NaN
# Justificación: Son claramente errores, mejor tratarlos como faltantes

# Edades menores a 10 o mayores a 100
for col in ['edadHombre', 'edadMujer']:
    if col in df_limpio.columns:
        # Contar antes
        antes = df_limpio[col].notna().sum()
        
        # Reemplazar valores imposibles
        df_limpio.loc[(df_limpio[col] < 10) | (df_limpio[col] > 100), col] = np.nan
        
        # Contar después
        despues = df_limpio[col].notna().sum()
        convertidos = antes - despues
        
        print(f"✓ {col}:")
        print(f"  - Valores convertidos a NaN: {convertidos:,}")
        print(f"  - Valores válidos restantes: {despues:,}")

print("\n✓ Tratamiento de edades completado")
print("="*70)


TRATAMIENTO DE EDADES PROBLEMÁTICAS:
✓ edadHombre:
  - Valores convertidos a NaN: 0
  - Valores válidos restantes: 26,354
✓ edadMujer:
  - Valores convertidos a NaN: 0
  - Valores válidos restantes: 26,491

✓ Tratamiento de edades completado


### 2.6. Validación de variables temporales

**Objetivo**: Verificar que fechas y años estén dentro de rangos esperados

In [47]:
# Validación de variables temporales
print("VALIDACIÓN DE VARIABLES TEMPORALES:")
print("="*70)

# Año de ocurrencia
if 'anioOcurrencia' in df_limpio.columns:
    print("\nAño de Ocurrencia:")
    print(f"  Rango: {df_limpio['anioOcurrencia'].min()} - {df_limpio['anioOcurrencia'].max()}")
    print(f"  Valores únicos: {df_limpio['anioOcurrencia'].nunique()}")
    
    # Verificar años fuera del rango esperado (2010-2020)
    fuera_rango = ((df_limpio['anioOcurrencia'] < 2010) | 
                (df_limpio['anioOcurrencia'] > 2020)).sum()
    print(f"  Años fuera de 2010-2020: {fuera_rango:,}")

# Mes de ocurrencia
if 'mesOcurrencia' in df_limpio.columns:
    print("\nMes de Ocurrencia:")
    print(f"  Rango: {df_limpio['mesOcurrencia'].min()} - {df_limpio['mesOcurrencia'].max()}")
    invalidos = ((df_limpio['mesOcurrencia'] < 1) | 
                (df_limpio['mesOcurrencia'] > 12)).sum()
    print(f"  Meses inválidos (< 1 o > 12): {invalidos:,}")

# Día de ocurrencia
if 'diaOcurrencia' in df_limpio.columns:
    print("\nDía de Ocurrencia:")
    print(f"  Rango: {df_limpio['diaOcurrencia'].min()} - {df_limpio['diaOcurrencia'].max()}")
    invalidos = ((df_limpio['diaOcurrencia'] < 1) | 
                (df_limpio['diaOcurrencia'] > 31)).sum()
    print(f"  Días inválidos (< 1 o > 31): {invalidos:,}")

print("="*70)

VALIDACIÓN DE VARIABLES TEMPORALES:

Año de Ocurrencia:
  Rango: 2010 - 2020
  Valores únicos: 11
  Años fuera de 2010-2020: 0

Mes de Ocurrencia:
  Rango: 1 - 12
  Meses inválidos (< 1 o > 12): 0

Día de Ocurrencia:
  Rango: 1 - 31
  Días inválidos (< 1 o > 31): 0


### 2.7. Estandarización de categorías

In [48]:
# Análisis de variables categóricas
print("ANÁLISIS DE VARIABLES CATEGÓRICAS:")
print("="*70)

variables_categoricas = ['grupoEtnicoHombre', 'grupoEtnicoMujer',
                        'nacionalidadHombre', 'nacionalidadMujer',
                        'escolaridadHombre', 'escolaridadMujer']

for var in variables_categoricas:
    if var in df_limpio.columns:
        print(f"\n{var}:")
        print(f"  Categorías únicas: {df_limpio[var].nunique()}")
        print(f"  Valores no nulos: {df_limpio[var].notna().sum():,}")
        print("  Top 5 categorías:")
        top5 = df_limpio[var].value_counts().head(5)
        for cat, count in top5.items():
            print(f"    {cat}: {count:,} ({count/len(df_limpio)*100:.1f}%)")

print("="*70)

ANÁLISIS DE VARIABLES CATEGÓRICAS:

grupoEtnicoHombre:
  Categorías únicas: 5
  Valores no nulos: 23,755
  Top 5 categorías:
    4.0: 17,210 (28.7%)
    1.0: 3,870 (6.5%)
    2.0: 2,438 (4.1%)
    5.0: 236 (0.4%)
    3.0: 1 (0.0%)

grupoEtnicoMujer:
  Categorías únicas: 5
  Valores no nulos: 23,704
  Top 5 categorías:
    4.0: 17,400 (29.0%)
    1.0: 3,580 (6.0%)
    2.0: 2,460 (4.1%)
    5.0: 263 (0.4%)
    3.0: 1 (0.0%)

nacionalidadHombre:
  Categorías únicas: 66
  Valores no nulos: 59,795
  Top 5 categorías:
    320.0: 58,795 (98.0%)
    840.0: 218 (0.4%)
    222.0: 146 (0.2%)
    720.0: 73 (0.1%)
    484.0: 50 (0.1%)

nacionalidadMujer:
  Categorías únicas: 53
  Valores no nulos: 59,785
  Top 5 categorías:
    320.0: 58,771 (98.0%)
    840.0: 246 (0.4%)
    222.0: 227 (0.4%)
    720.0: 85 (0.1%)
    340.0: 68 (0.1%)

escolaridadHombre:
  Categorías únicas: 7
  Valores no nulos: 27,137
  Top 5 categorías:
    4.0: 8,595 (14.3%)
    1.0: 7,290 (12.2%)
    2.0: 4,569 (7.6%)
    3.0: 

In [49]:
# Análisis de ocupaciones (pueden tener muchas categorías)
print("\nANÁLISIS DE OCUPACIONES:")
print("="*70)

for var in ['ocupacionHombre', 'ocupacionMujer']:
    if var in df_limpio.columns:
        print(f"\n{var}:")
        print(f"  Categorías únicas: {df_limpio[var].nunique()}")
        print(f"  Valores no nulos: {df_limpio[var].notna().sum():,}")
        print("  Top 10 ocupaciones:")
        top10 = df_limpio[var].value_counts().head(10)
        for i, (cat, count) in enumerate(top10.items(), 1):
            print(f"    {i}. {cat}: {count:,} ({count/len(df_limpio)*100:.1f}%)")

print("="*70)


ANÁLISIS DE OCUPACIONES:

ocupacionHombre:
  Categorías únicas: 379
  Valores no nulos: 32,357
  Top 10 ocupaciones:
    1. 92.0: 4,254 (7.1%)
    2. 9712.0: 2,229 (3.7%)
    3. 52.0: 2,076 (3.5%)
    4. 9999: 1,610 (2.7%)
    5. 9712: 1,423 (2.4%)
    6. 43.0: 1,291 (2.2%)
    7. 61: 1,148 (1.9%)
    8. 23.0: 907 (1.5%)
    9. 6111.0: 880 (1.5%)
    10. 72.0: 831 (1.4%)

ocupacionMujer:
  Categorías únicas: 248
  Valores no nulos: 20,228
  Top 10 ocupaciones:
    1. 9712.0: 2,119 (3.5%)
    2. 23.0: 1,864 (3.1%)
    3. 9999: 1,822 (3.0%)
    4. 9711.0: 1,639 (2.7%)
    5. 41.0: 1,504 (2.5%)
    6. 9712: 1,336 (2.2%)
    7. 9711: 1,048 (1.7%)
    8. 43.0: 781 (1.3%)
    9. 52.0: 631 (1.1%)
    10. 23: 552 (0.9%)


Las categorías están codificadas numéricamente por el INE. Se mantendrán tal cual para el análisis exploratorio. En fases posteriores, se crearán diccionarios de mapeo a etiquetas descriptivas.

### 2.8. Dataset final

**Decisiones clave**:
- **No se eliminaron registros**: Se mantiene la completitud del dataset
- **Valores faltantes preservados**: Permiten análisis sobre patrones de datos faltantes
- **Tipos de datos optimizados**: Int64 (maneja NaN) y category (eficiente en memoria)

In [50]:
# Resumen final del dataset limpio
print("="*70)
print("DATASET FINAL LIMPIO - DIVORCIOS GUATEMALA 2010-2020")
print("="*70)
print("\nDIMENSIONES:")
print(f"   Registros: {len(df_limpio):,}")
print(f"   Variables: {len(df_limpio.columns)}")

print("\nMEMORIA:")
print(f"   Uso total: {df_limpio.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nCOMPLETITUD GENERAL:")
total_valores = df_limpio.shape[0] * df_limpio.shape[1]
valores_presentes = df_limpio.notna().sum().sum()
completitud_total = (valores_presentes / total_valores * 100)
print(f"   Valores presentes: {valores_presentes:,} de {total_valores:,}")
print(f"   Completitud global: {completitud_total:.2f}%")

print("\nVARIABLES CLAVE:")
vars_clave = ['edadHombre', 'edadMujer', 'anioOcurrencia', 
            'departamentoOcurrencia', 'grupoEtnicoHombre', 'grupoEtnicoMujer']
for var in vars_clave:
    if var in df_limpio.columns:
        completitud_var = (df_limpio[var].notna().sum() / len(df_limpio) * 100)
        print(f"   {var}: {completitud_var:.1f}% completo")

print("\n" + "="*70)
print("Dataset limpio y listo para análisis exploratorio :)")
print("="*70)

DATASET FINAL LIMPIO - DIVORCIOS GUATEMALA 2010-2020

DIMENSIONES:
   Registros: 59,994
   Variables: 19

MEMORIA:
   Uso total: 4.63 MB

COMPLETITUD GENERAL:
   Valores presentes: 867,092 de 1,139,886
   Completitud global: 76.07%

VARIABLES CLAVE:
   edadHombre: 43.9% completo
   edadMujer: 44.2% completo
   anioOcurrencia: 100.0% completo
   departamentoOcurrencia: 100.0% completo
   grupoEtnicoHombre: 39.6% completo
   grupoEtnicoMujer: 39.5% completo

Dataset limpio y listo para análisis exploratorio :)


In [51]:
# Vista previa del dataset limpio
print("\nVISTA PREVIA DEL DATASET LIMPIO:")
print("="*100)
df_limpio.head(10)


VISTA PREVIA DEL DATASET LIMPIO:


,departamentoRegistro,municipioRegistro,mesRegistro,anioRegistro,diaOcurrencia,mesOcurrencia,anioOcurrencia,departamentoOcurrencia,municipioOcurrencia,edadHombre,edadMujer,grupoEtnicoHombre,grupoEtnicoMujer,nacionalidadHombre,nacionalidadMujer,escolaridadHombre,escolaridadMujer,ocupacionHombre,ocupacionMujer
0,1.0,0101,8,2010,4,5,2010,1.0,0101,41,35,NaN,NaN,320.0,320.0,NaN,NaN,1120.0,1120.0
1,1.0,0101,11,2010,6,10,2010,1.0,0101,<NA>,<NA>,NaN,NaN,320.0,320.0,NaN,NaN,2161.0,1219.0
2,1.0,0101,3,2010,16,2,2010,1.0,0101,42,31,NaN,NaN,320.0,320.0,4.0,NaN,2166.0,1219.0
3,1.0,0101,10,2010,25,8,2010,1.0,0101,47,36,NaN,NaN,320.0,320.0,1.0,1.0,5221.0,1219.0
4,1.0,0101,8,2010,20,5,2010,1.0,0101,<NA>,<NA>,NaN,NaN,840.0,484.0,NaN,NaN,9712.0,1219.0
5,16.0,1601,3,2010,28,1,2010,16.0,1601,23,37,2.0,2.0,320.0,320.0,NaN,3.0,9712.0,1221.0
6,1.0,0101,10,2010,23,6,2010,1.0,0101,<NA>,<NA>,NaN,NaN,320.0,320.0,NaN,NaN,5249.0,1346.0
7,1.0,0101,11,2010,24,8,2010,1.0,0101,<NA>,<NA>,NaN,NaN,320.0,320.0,NaN,NaN,1219.0,1411.0
8,1.0,0101,9,2010,15,7,2010,1.0,0101,<NA>,<NA>,2.0,2.0,320.0,320.0,5.0,5.0,2113.0,2131.0
9,1.0,0101,11,2010,8,9,2010,1.0,0101,33,33,NaN,NaN,320.0,320.0,NaN,NaN,2611.0,2141.0


In [52]:
# Información detallada del dataset
print("\nINFORMACIÓN DETALLADA DEL DATASET:")
print("="*70)
df_limpio.info()
print("="*70)


INFORMACIÓN DETALLADA DEL DATASET:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59994 entries, 0 to 59993
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   departamentoRegistro    59994 non-null  category
 1   municipioRegistro       59994 non-null  category
 2   mesRegistro             59994 non-null  Int64   
 3   anioRegistro            59994 non-null  Int64   
 4   diaOcurrencia           59994 non-null  Int64   
 5   mesOcurrencia           59994 non-null  Int64   
 6   anioOcurrencia          59994 non-null  Int64   
 7   departamentoOcurrencia  59994 non-null  category
 8   municipioOcurrencia     59994 non-null  category
 9   edadHombre              26354 non-null  Int64   
 10  edadMujer               26491 non-null  Int64   
 11  grupoEtnicoHombre       23755 non-null  category
 12  grupoEtnicoMujer        23704 non-null  category
 13  nacionalidadHombre      59795 non-null  

## 3. Descripción Formal del Dataset

Esta sección presenta una caracterización técnica del dataset consolidado de divorcios en Guatemala para el período 2010-2020, posterior al proceso de limpieza y estandarización. La descripción incluye dimensiones del conjunto de datos, taxonomía de variables y documentación semántica.

### 3.1. Dimensiones del Dataset

In [53]:
# Dimensiones del dataset consolidado
n_observaciones = len(df_limpio)
n_variables = len(df_limpio.columns)

print("="*70)
print("CARACTERIZACIÓN DIMENSIONAL DEL DATASET")
print("="*70)
print(f"\nObservaciones (n): {n_observaciones:,}")
print(f"Variables (p): {n_variables}")
print(f"\n   Razón n/p: {n_observaciones/n_variables:,.1f}")
print(f"   Total de valores: {n_observaciones * n_variables:,}")
print("="*70)

CARACTERIZACIÓN DIMENSIONAL DEL DATASET

Observaciones (n): 59,994
Variables (p): 19

   Razón n/p: 3,157.6
   Total de valores: 1,139,886


### 3.2. Taxonomía de Variables

Las variables del dataset se clasifican según los siguientes criterios estadísticos:

1. **Variables Numéricas Discretas**: Valores enteros que representan conteos o categorías ordinales
2. **Variables Numéricas Continuas**: En este dataset no aplica, todas las numéricas son discretas
3. **Variables Categóricas Nominales**: Categorías sin orden inherente
4. **Variables Categóricas Ordinales**: Categorías con orden jerárquico

**Nota**: Las variables de ocupación, aunque codificadas numéricamente según clasificación CIUO (Clasificación Internacional Uniforme de Ocupaciones), se tratan como categóricas nominales debido a que los códigos no representan un orden intrínseco.

In [54]:
# Clasificación taxonómica de variables
import pandas as pd

# Definir taxonomía de variables
taxonomia = {
    # Variables temporales (numéricas discretas)
    'diaOcurrencia': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Día del mes en que ocurrió el divorcio (1-31)',
        'categoria': 'Temporal'
    },
    'mesOcurrencia': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Mes del año en que ocurrió el divorcio (1-12)',
        'categoria': 'Temporal'
    },
    'anioOcurrencia': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Año en que ocurrió el divorcio (2010-2020)',
        'categoria': 'Temporal'
    },
    'mesRegistro': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Mes en que se registró el divorcio (1-12)',
        'categoria': 'Temporal'
    },
    'anioRegistro': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Año en que se registró el divorcio',
        'categoria': 'Temporal'
    },
    
    # Variables geográficas (categóricas nominales)
    'departamentoRegistro': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código del departamento donde se registró el divorcio',
        'categoria': 'Geográfica'
    },
    'municipioRegistro': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código del municipio donde se registró el divorcio',
        'categoria': 'Geográfica'
    },
    'departamentoOcurrencia': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código del departamento donde ocurrió el divorcio',
        'categoria': 'Geográfica'
    },
    'municipioOcurrencia': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código del municipio donde ocurrió el divorcio',
        'categoria': 'Geográfica'
    },
    
    # Variables demográficas - Edad (numéricas discretas)
    'edadHombre': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Edad del esposo al momento del divorcio (años completos)',
        'categoria': 'Demográfica'
    },
    'edadMujer': {
        'tipo': 'Numérica Discreta',
        'descripcion': 'Edad de la esposa al momento del divorcio (años completos)',
        'categoria': 'Demográfica'
    },
    
    # Variables demográficas - Grupo étnico (categóricas nominales)
    'grupoEtnicoHombre': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de autoidentificación étnica del esposo',
        'categoria': 'Demográfica'
    },
    'grupoEtnicoMujer': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de autoidentificación étnica de la esposa',
        'categoria': 'Demográfica'
    },
    
    # Variables demográficas - Nacionalidad (categóricas nominales)
    'nacionalidadHombre': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de nacionalidad del esposo',
        'categoria': 'Demográfica'
    },
    'nacionalidadMujer': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de nacionalidad de la esposa',
        'categoria': 'Demográfica'
    },
    
    # Variables socio-educativas - Escolaridad (categóricas ordinales)
    'escolaridadHombre': {
        'tipo': 'Categórica Ordinal',
        'descripcion': 'Nivel máximo de escolaridad alcanzado por el esposo',
        'categoria': 'Socio-educativa'
    },
    'escolaridadMujer': {
        'tipo': 'Categórica Ordinal',
        'descripcion': 'Nivel máximo de escolaridad alcanzado por la esposa',
        'categoria': 'Socio-educativa'
    },
    
    # Variables socio-económicas - Ocupación (categóricas nominales)
    'ocupacionHombre': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de ocupación del esposo según clasificación CIUO',
        'categoria': 'Socio-económica'
    },
    'ocupacionMujer': {
        'tipo': 'Categórica Nominal',
        'descripcion': 'Código de ocupación de la esposa según clasificación CIUO',
        'categoria': 'Socio-económica'
    }
}

# Crear DataFrame de taxonomía
df_taxonomia = pd.DataFrame.from_dict(taxonomia, orient='index')
df_taxonomia.index.name = 'Variable'
df_taxonomia = df_taxonomia.reset_index()

print("Taxonomía de variables definida correctamente")
print(f"Total de variables clasificadas: {len(df_taxonomia)}")

Taxonomía de variables definida correctamente
Total de variables clasificadas: 19


In [55]:
# Resumen cuantitativo por tipo de variable
print("="*70)
print("DISTRIBUCIÓN DE VARIABLES POR TIPO")
print("="*70)

# Contar por tipo
conteo_tipos = df_taxonomia['tipo'].value_counts()
print("\n- Por tipo estadístico:")
for tipo, count in conteo_tipos.items():
    porcentaje = (count / len(df_taxonomia)) * 100
    print(f"   • {tipo}: {count} ({porcentaje:.1f}%)")

# Contar por categoría semántica
print("\n- Por categoría semántica:")
conteo_categorias = df_taxonomia['categoria'].value_counts()
for cat, count in conteo_categorias.items():
    porcentaje = (count / len(df_taxonomia)) * 100
    print(f"   • {cat}: {count} ({porcentaje:.1f}%)")

print("="*70)

DISTRIBUCIÓN DE VARIABLES POR TIPO

- Por tipo estadístico:
   • Categórica Nominal: 10 (52.6%)
   • Numérica Discreta: 7 (36.8%)
   • Categórica Ordinal: 2 (10.5%)

- Por categoría semántica:
   • Demográfica: 6 (31.6%)
   • Temporal: 5 (26.3%)
   • Geográfica: 4 (21.1%)
   • Socio-educativa: 2 (10.5%)
   • Socio-económica: 2 (10.5%)


### 3.3. Tabla Descriptiva de Variables

La siguiente tabla presenta la documentación completa de cada variable, incluyendo su identificador, tipo estadístico, categoría semántica y descripción operacional.

In [56]:
# Mostrar tabla completa de variables
print("\n" + "="*100)
print("TABLA DESCRIPTIVA DE VARIABLES DEL DATASET")
print("="*100)
print()

# Configurar pandas para mostrar toda la tabla
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Mostrar tabla ordenada por categoría
df_taxonomia_ordenada = df_taxonomia.sort_values(['categoria', 'Variable'])
print(df_taxonomia_ordenada.to_string(index=False))

print("\n" + "="*100)

# Restablecer opciones
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')
pd.reset_option('display.width')


TABLA DESCRIPTIVA DE VARIABLES DEL DATASET

              Variable               tipo                                                descripcion       categoria
            edadHombre  Numérica Discreta   Edad del esposo al momento del divorcio (años completos)     Demográfica
             edadMujer  Numérica Discreta Edad de la esposa al momento del divorcio (años completos)     Demográfica
     grupoEtnicoHombre Categórica Nominal             Código de autoidentificación étnica del esposo     Demográfica
      grupoEtnicoMujer Categórica Nominal           Código de autoidentificación étnica de la esposa     Demográfica
    nacionalidadHombre Categórica Nominal                          Código de nacionalidad del esposo     Demográfica
     nacionalidadMujer Categórica Nominal                        Código de nacionalidad de la esposa     Demográfica
departamentoOcurrencia Categórica Nominal          Código del departamento donde ocurrió el divorcio      Geográfica
  departamentoRegis

### 3.4. Características Operacionales del Dataset

**Propiedades estructurales:**

1. **Granularidad**: Cada observación representa un evento de divorcio único registrado entre 2010-2020
2. **Temporalidad**: Dataset de corte transversal con componente temporal agregado
3. **Completitud**: Dataset con valores faltantes (estrategia: conservación sin imputación)
4. **Codificación**: Variables categóricas codificadas según estándares del INE de Guatemala

In [58]:
# Resumen estadístico por tipo de variable

print("="*70)
print("RESUMEN ESTADÍSTICO POR TIPO DE VARIABLE")
print("="*70)

# Variables numéricas discretas
vars_numericas = ['diaOcurrencia', 'mesOcurrencia', 'anioOcurrencia', 
                'mesRegistro', 'anioRegistro', 'edadHombre', 'edadMujer']

print("\n1. VARIABLES NUMÉRICAS DISCRETAS:")
print("-" * 70)
for var in vars_numericas:
    if var in df_limpio.columns:
        print(f"\n   {var}:")
        print(f"      Rango: [{df_limpio[var].min()}, {df_limpio[var].max()}]")
        print(f"      Media: {df_limpio[var].mean():.2f}")
        print(f"      Mediana: {df_limpio[var].median():.1f}")
        print(f"      Desv. Est.: {df_limpio[var].std():.2f}")
        print(f"      Valores únicos: {df_limpio[var].nunique()}")

print("\n" + "="*70)

RESUMEN ESTADÍSTICO POR TIPO DE VARIABLE

1. VARIABLES NUMÉRICAS DISCRETAS:
----------------------------------------------------------------------

   diaOcurrencia:
      Rango: [1, 31]
      Media: 15.72
      Mediana: 16.0
      Desv. Est.: 8.69
      Valores únicos: 31

   mesOcurrencia:
      Rango: [1, 12]
      Media: 6.32
      Mediana: 6.0
      Desv. Est.: 3.31
      Valores únicos: 12

   anioOcurrencia:
      Rango: [2010, 2020]
      Media: 2015.36
      Mediana: 2016.0
      Desv. Est.: 3.02
      Valores únicos: 11

   mesRegistro:
      Rango: [1, 12]
      Media: 6.57
      Mediana: 7.0
      Desv. Est.: 3.43
      Valores únicos: 12

   anioRegistro:
      Rango: [2010, 2021]
      Media: 2015.59
      Mediana: 2016.0
      Desv. Est.: 3.05
      Valores únicos: 12

   edadHombre:
      Rango: [15, 96]
      Media: 35.51
      Mediana: 33.0
      Desv. Est.: 10.21
      Valores únicos: 76

   edadMujer:
      Rango: [14, 80]
      Media: 32.16
      Mediana: 30.0
    

In [59]:
# Resumen de variables categóricas

print("="*70)
print("RESUMEN DE VARIABLES CATEGÓRICAS")
print("="*70)

# Variables categóricas por grupo
grupos_categoricas = {
    'Geográficas': ['departamentoRegistro', 'municipioRegistro', 
                    'departamentoOcurrencia', 'municipioOcurrencia'],
    'Demográficas (Nominal)': ['grupoEtnicoHombre', 'grupoEtnicoMujer',
                               'nacionalidadHombre', 'nacionalidadMujer'],
    'Socio-educativas (Ordinal)': ['escolaridadHombre', 'escolaridadMujer'],
    'Socio-económicas (Nominal)': ['ocupacionHombre', 'ocupacionMujer']
}

for grupo, variables in grupos_categoricas.items():
    print(f"\n{grupo}:")
    print("-" * 70)
    for var in variables:
        if var in df_limpio.columns:
            n_categorias = df_limpio[var].nunique()
            n_validos = df_limpio[var].notna().sum()
            completitud = (n_validos / len(df_limpio)) * 100
            print(f"   • {var}:")
            print(f"      Categorías únicas: {n_categorias}")
            print(f"      Registros válidos: {n_validos:,} ({completitud:.1f}%)")

print("\n" + "="*70)

RESUMEN DE VARIABLES CATEGÓRICAS

Geográficas:
----------------------------------------------------------------------
   • departamentoRegistro:
      Categorías únicas: 22
      Registros válidos: 59,994 (100.0%)
   • municipioRegistro:
      Categorías únicas: 336
      Registros válidos: 59,994 (100.0%)
   • departamentoOcurrencia:
      Categorías únicas: 22
      Registros válidos: 59,994 (100.0%)
   • municipioOcurrencia:
      Categorías únicas: 337
      Registros válidos: 59,994 (100.0%)

Demográficas (Nominal):
----------------------------------------------------------------------
   • grupoEtnicoHombre:
      Categorías únicas: 5
      Registros válidos: 23,755 (39.6%)
   • grupoEtnicoMujer:
      Categorías únicas: 5
      Registros válidos: 23,704 (39.5%)
   • nacionalidadHombre:
      Categorías únicas: 66
      Registros válidos: 59,795 (99.7%)
   • nacionalidadMujer:
      Categorías únicas: 53
      Registros válidos: 59,785 (99.7%)

Socio-educativas (Ordinal):
-------